In [ ]:
import pandas as pd
import numpy as np
import os
import re
import datetime as dt

In [ ]:
# Словарь для транслитерации и функции для преобразования букв в большие
dic = {'ь':'', 'ъ':'', 'а':'a', 'б':'b','в':'v',
       'г':'g', 'д':'d', 'е':'e', 'ё':'yo','ж':'zh',
       'з':'z', 'и':'i', 'й':'j', 'к':'k', 'л':'l',
       'м':'m', 'н':'n', 'о':'o', 'п':'p', 'р':'r', 
       'с':'s', 'т':'t', 'у':'u', 'ф':'f', 'х':'kh',
       'ц':'ts', 'ч':'ch', 'ш':'sh', 'щ':'sch', 'ы':'y',
       'э':'e', 'ю':'yu', 'я':'ya'}
 
def tr(x):
    t = ''
    for i in x:
        t+=dic.get(i.lower(), i.lower()).upper() if i.isupper() else dic.get(i, i)
    return t
def tr2(x):
    t = ''
    for i in x:
        t+=dic.get(i.lower(), i.lower()).capitalize() if i.isupper() else dic.get(i, i)
    return t

In [ ]:
for filename in os.listdir('./inputsson'):
    if filename.endswith('.csv'):
        df = pd.read_csv('./inputsson/' + filename, sep=';', encoding='cp1251', dtype=({'INN':str, 'DT':str}), error_bad_lines=False, warn_bad_lines=True)
df=df.drop(columns=['Unnamed: 9'])
df.columns = ['Дата', 'Название', 'ИНН', 'Адрес', 'Сайт', 'Признаки, установленные Банком России', 'Регионы', 'Деятельность прекращена', 'Дополнительно']

In [ ]:
for i in range(df['Дата'].shape[0]):
    df['Дата'].iloc[i] = dt.datetime.strptime(df['Дата'].iloc[i], '%d.%m.%Y')

In [ ]:
date_str = input("Введите дату, с которой формируются списки, в формате dd.mm.yyyy\n")
new_date = dt.datetime.strptime(date_str, '%d.%m.%Y')
df=df.loc[df['Дата'] >= new_date]

In [ ]:
# Список ИНН
df_INN = df['ИНН'].dropna().reset_index(drop = True)
df_INN.to_csv('202_INN.csv', sep=';', encoding='cp1251', index=False)

In [ ]:
# Список названий мерчантов с наличием адресов
df_address = df[['Адрес', 'Название']]
df_address = df_address.loc[pd.notnull(df_address['Адрес'])]
# Разбиваем названия по запятым
df_name = df_address['Название'].str.split(', ', expand=True)
# Сводим с адресами
df_address = pd.concat([df_address['Адрес'], df_name], axis = 1)
# Названия одним столбцом
df_address = df_address.set_index('Адрес').stack().reset_index(name='Имя').drop('level_1',axis='columns')

In [ ]:
# Функция для получения имени в кавычках
def quotes(df0):
    for i in range(df0.shape[0]):
        row = df0.iloc[i]
        if re.search('".+".+".+".+".+".+"(.+)"', row):
            df0.iloc[i] = re.search('".+".+".+".+".+".+"(.+)"', row).group(1)
        elif re.search('".+".+".+".+".+"(.+)"', row):
            df0.iloc[i] = re.search('".+".+".+".+".+"(.+)"', row).group(1)
        elif re.search('".+".+".+".+"(.+)"', row):
            df0.iloc[i] = re.search('".+".+".+".+"(.+)"', row).group(1)
        elif re.search('".+".+".+"(.+)"', row):
            df0.iloc[i] = re.search('".+".+".+"(.+)"', row).group(1)
        elif re.search('".+".+"(.+)"', row):
            df0.iloc[i] = re.search('".+".+"(.+)"', row).group(1)
        elif re.search('".+"(.+)"', row):
            df0.iloc[i] = re.search('".+"(.+)"', row).group(1)
        elif re.search('["|«](.+)["|»]', row):
            df0.iloc[i] = re.search('["|«](.+)["|»]', row).group(1)
    return(df0)

In [ ]:
# Получаем значения имен из кавычек
quotes(df_address['Имя'])
#Транслитерируем имена мерчантов
for i in range(df_address['Имя'].shape[0]):
    if re.search('[А-ЯЁ]{4}', df_address['Имя'].iloc[i]):
        df_address['Имя'].iloc[i] = tr(df_address['Имя'].iloc[i])
    else:
        df_address['Имя'].iloc[i] = tr2(df_address['Имя'].iloc[i])

In [ ]:
# Вставляем столбцы Индекс и Город
df_address.insert(1, 'Индекс', None)
df_address.insert(2, 'Город', 'FILL ' + df_address['Адрес'])
# Получаем значения индексов
for i in range(df_address['Адрес'].shape[0]):
    row = df_address['Адрес'].iloc[i]
    if re.match('(\d{6}),', row):
        df_address['Индекс'].iloc[i] = re.match('^(\d{6}),', row).group(1)
# # Убираем строки, в которых индекс отсутствует, для списка с индексами
# df_index = df_address.loc[pd.notnull(df_address['Индекс'])]
# df_index = df_index.drop_duplicates().reset_index(drop=True)
# Получаем значения городов и сел
for i in range(df_address['Адрес'].shape[0]):
    row = df_address['Адрес'].iloc[i]
    if re.search(',\s{1,}[^\.]*[г|с|п|д|Г|С|П|Д]\.\s([^,\d\.]+),', row):
        df_address['Город'].iloc[i] = re.search(',\s{1,}[^\.]*[г|с|п|д|Г|С|П|Д]\.\s([^,\d\.]+),', row).group(1)
# Транслитерируем города
    if re.search('[А-ЯЁ]{4}', df_address['Город'].iloc[i]):
        df_address['Город'].iloc[i] = tr(df_address['Город'].iloc[i])
    else:
        df_address['Город'].iloc[i] = tr2(df_address['Город'].iloc[i])
#Удаляем дубликаты
df_address = df_address.drop_duplicates().reset_index(drop=True)

In [ ]:
# Имена мерчантов без условий на адрес и индекс, не вошедшие в предыдущий список, и все сайты
df_names = df[~df['Адрес'].isin(df_address['Адрес'])]
df_names = pd.concat([df_names['Название'], df['Сайт'].loc[pd.notnull(df['Сайт'])]], axis = 0)
df_names = df_names.str.split(', ', expand=True)
df_names = df_names.stack().reset_index(name='Имя').drop(['level_0', 'level_1'],axis='columns')
# Получаем значения имен из кавычек
quotes(df_names['Имя'])
# Транслитерируем имена мерчантов
for i in range(df_names['Имя'].shape[0]):
    if re.search('[А-ЯЁ]{4}', df_names['Имя'].iloc[i]):
        df_names['Имя'].iloc[i] = tr(df_names['Имя'].iloc[i])
    else:
        df_names['Имя'].iloc[i] = tr2(df_names['Имя'].iloc[i])
# #Убираем домены из сайтов
#     if re.search('\.(ru|kz|su|rf)$', df_names['Имя'].iloc[i].lower()):
#         df_names['Имя'].iloc[i] = re.sub('\.(ru|kz|su|rf)$', '', df_names['Имя'].iloc[i].lower())
#     elif re.search('\.(com|org|net|biz)$', df_names['Имя'].iloc[i].lower()):
#         df_names['Имя'].iloc[i] = re.sub('\.(com|org|net|biz)$', '', df_names['Имя'].iloc[i].lower())
#     if re.search('^(www)\.', df_names['Имя'].iloc[i].lower()):
#         df_names['Имя'].iloc[i] = re.sub('^(www)\.', '', df_names['Имя'].iloc[i].lower())
#Удаляем дубликаты
df_names = df_names.drop_duplicates().reset_index(drop=True)

In [ ]:
#Убираем пробелы в начале строк
df_names['Имя'].loc[df_names['Имя'].str[0]==' '] = df_names['Имя'].str[1:]
# df_index['Имя'].loc[df_index['Имя'].str[0]==' '] = df_index['Имя'].str[1:]
# df_index['Индекс'].loc[df_index['Индекс'].str[0]==' '] = df_index['Индекс'].str[1:]
df_address['Имя'].loc[df_address['Имя'].str[0]==' '] = df_address['Имя'].str[1:]
df_address['Город'].loc[df_address['Город'].str[0]==' '] = df_address['Город'].str[1:]
df_address['Индекс'].loc[df_address['Индекс'].str[0]==' '] = df_address['Индекс'].str[1:]

In [ ]:
#Убираем символы переноса строки
df_names['Имя'] = df_names['Имя'].str.rstrip('\n\r')
df_address['Имя'] = df_address['Имя'].str.rstrip('\n\r')
df_address['Город'] = df_address['Город'].str.rstrip('\n\r')
df_address['Индекс'] = df_address['Индекс'].str.rstrip('\n\r')

In [ ]:
#Делаем списки для ручной корректировки
df_names.to_csv('201_Names_EPOS.csv', sep=';', encoding='cp1251', index=False)
# df_index[['Имя', 'Индекс']].to_csv('203_Index.csv', sep=';', encoding='cp1251', index=False)
df_address.drop(columns='Адрес').to_csv('203_205_Index_Cities.csv', sep=';', encoding='cp1251', index=False)

In [ ]:
with open("Журнал обновления списков.txt", "a") as file:
    file.write('\nПоследняя запись в списке ЦБ добавлена ' + df['Дата'].max().strftime('%d.%m.%Y'))